# Agent Tools and Configurations

Tools give agents ability to take actions: sequential/parallel calls, dynamic selection, error handling, and state persistence.

In [ ]:
import sys
sys.path.append('D:/Courses/Udemy/AI Agent Projects')

import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from scripts import base_tools

In [ ]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

## Basic Agent with Tools

In [ ]:
agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather]
)

response = agent.invoke({
    'messages': [HumanMessage("What's the weather in Tokyo?")]
})

response['messages'][-1].text

In [ ]:
# Agent without tools - single LLM node
agent_no_tools = create_agent(model=model, tools=[])

response = agent_no_tools.invoke({
    'messages': [HumanMessage("Hello")]
})

response['messages'][-1].text

## Tool Error Handling

In [ ]:
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage

@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        return ToolMessage(
            content=f"Tool error: {str(e)}",
            tool_call_id=request.tool_call["id"]
        )

agent_safe = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    middleware=[handle_tool_errors]
)

response = agent_safe.invoke({
    'messages': [HumanMessage("Search for AI news")]
})

response['messages'][-1].text

## Sequential Tool Calls

In [ ]:
# Agent calls tools in sequence
response = agent.invoke({
    'messages': [HumanMessage(
        "Search for Apple news, then tell me weather in Cupertino"
    )]
})

response['messages'][-1].text

## Parallel Tool Calls

In [ ]:
# Agent calls multiple tools simultaneously
response = agent.invoke({
    'messages': [HumanMessage(
        "What's the weather in Paris and London?"
    )]
})

response['messages'][-1].text

## Dynamic Tool Selection

In [ ]:
# Agent chooses appropriate tool based on query
agent_smart = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt="Choose the right tool for each query."
)

queries = [
    "Latest tech news",
    "Temperature in Mumbai",
    "Microsoft stock price"
]

for q in queries:
    response = agent_smart.invoke({'messages': [HumanMessage(q)]})
    print(f"Q: {q}")
    print(f"A: {response['messages'][-1].text[:100]}...\n")

## Key Takeaways

- Tools extend agent capabilities with actions
- `@wrap_tool_call` handles errors gracefully
- Sequential and parallel execution based on context
- Agents dynamically select appropriate tools
- Clear docstrings guide tool selection

In [ ]:
# Exercise: Test with different queries
